DATASET

In [1]:
from torch_geometric_temporal.dataset import METRLADatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

In [2]:
loader = METRLADatasetLoader()

In [3]:
dataset = loader.get_dataset()

In [5]:
# Split the dataset into train and test sets
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

In [ ]:
for time, snapshot in enumerate(train_dataset):
    print(f"Snapshot {time}")
    print(f"x: {snapshot.x.shape}")
    print(f"edge_index: {snapshot.edge_index.shape}")
    print(f"edge_attr: {snapshot.edge_attr.shape}")
    print(f"y: {snapshot.y.shape}")
    break  # Print only the first snapshot

In [22]:
import networkx as nx

In [23]:
def show_graph_with_labels(adjacency_matrix, mylabels, data, count):
    rows, cols = np.where(adjacency_matrix>0)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    nx.draw(gr, node_size=500, with_labels=True, node_color="tab:cyan")
    plt.title(f'{data} ({count}) graph learnt from 30 epochs ')
    plt.show()

In [ ]:
path = 'adj_mat.npy'
adj = np.load(path)
gr=nx.from_numpy_array(adj)
print("Traffic graph learnt has {} nodes and {} edges".format(gr.number_of_nodes(),gr.number_of_edges()))
show_graph_with_labels(adj[0:100,0:100], range(1,101), "Traffic Roads", 100)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric_temporal.dataset import METRLADatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

# Check for device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Define the necessary sub-components

class nconv(nn.Module):
    def __init__(self):
        super(nconv, self).__init__()

    def forward(self, x, A):
        x = torch.einsum('ncwl,vw->ncvl', (x, A))
        return x.contiguous()

class linear(nn.Module):
    def __init__(self, c_in, c_out, bias=True):
        super(linear, self).__init__()
        self.mlp = nn.Conv2d(c_in, c_out, kernel_size=(1, 1), padding=(0, 0), stride=(1, 1), bias=bias)

    def forward(self, x):
        return self.mlp(x)

class mixprop(nn.Module):
    def __init__(self, c_in, c_out, gdep, dropout, alpha):
        super(mixprop, self).__init__()
        self.nconv = nconv()
        self.mlp = linear((gdep + 1) * c_in, c_out)
        self.gdep = gdep
        self.dropout = dropout
        self.alpha = alpha

    def forward(self, x, adj):
        adj = adj + torch.eye(adj.size(0)).to(x.device)
        d = adj.sum(1)
        h = x
        out = [h]
        a = adj / d.view(-1, 1)
        for i in range(self.gdep):
            h = self.alpha * x + (1 - self.alpha) * self.nconv(h, a)
            out.append(h)
        ho = torch.cat(out, dim=1)
        ho = self.mlp(ho)
        return ho

class dilated_inception(nn.Module):
    def __init__(self, cin, cout, dilation_factor=2, kernel_set=[2, 3, 6, 7]):
        super(dilated_inception, self).__init__()
        self.tconv = nn.ModuleList()
        self.kernel_set = kernel_set
        cout = int(cout / len(self.kernel_set))
        for kern in self.kernel_set:
            self.tconv.append(nn.Conv2d(cin, cout, (1, kern), dilation=(1, dilation_factor)))

    def forward(self, input):
        time_steps = input.size(3)  # Get the time dimension size
        x = []
        for i in range(len(self.kernel_set)):
            kernel_size = self.kernel_set[i]
            # Check if kernel size is less than or equal to input sequence length
            if kernel_size <= time_steps:
                x.append(self.tconv[i](input))
            else:
                # Skip kernels larger than the sequence length to avoid runtime errors
                print(f"Skipping kernel size {kernel_size} for sequence length {time_steps}")
        # Ensure each output has the same length in the time dimension
        for i in range(len(x)):
            x[i] = x[i][..., -x[-1].size(3):]
        x = torch.cat(x, dim=1)
        return x


class graph_constructor(nn.Module):
    def __init__(self, nnodes, k, dim, device, alpha=3, static_feat=None):
        super(graph_constructor, self).__init__()
        self.nnodes = nnodes
        self.emb1 = nn.Embedding(nnodes, dim)
        self.emb2 = nn.Embedding(nnodes, dim)
        self.lin1 = nn.Linear(dim, dim)
        self.lin2 = nn.Linear(dim, dim)
        self.device = device
        self.k = k
        self.alpha = alpha

    def forward(self, idx):
        nodevec1 = self.emb1(idx)
        nodevec2 = self.emb2(idx)
        nodevec1 = torch.tanh(self.alpha * self.lin1(nodevec1))
        nodevec2 = torch.tanh(self.alpha * self.lin2(nodevec2))
        a = torch.mm(nodevec1, nodevec2.transpose(1, 0)) - torch.mm(nodevec2, nodevec1.transpose(1, 0))
        adj = F.relu(torch.tanh(self.alpha * a))
        mask = torch.zeros(idx.size(0), idx.size(0)).to(self.device)
        mask.fill_(float('0'))
        s1, t1 = (adj + torch.rand_like(adj) * 0.01).topk(self.k, 1)
        mask.scatter_(1, t1, s1.fill_(1))
        adj = adj * mask
        return adj

# Define the CustomMTGNN model
class CustomMTGNN(nn.Module):
    def __init__(self, num_nodes, input_dim, output_dim, gdep, dropout, alpha, dilation_factor):
        super(CustomMTGNN, self).__init__()
        self.graph_constructor = graph_constructor(nnodes=num_nodes, k=20, dim=40, device=device)
        self.mixprop = mixprop(c_in=input_dim, c_out=output_dim, gdep=gdep, dropout=dropout, alpha=alpha)
        self.dilated_inception = dilated_inception(cin=output_dim, cout=output_dim, dilation_factor=dilation_factor)
        self.output_layer = nn.Conv2d(output_dim, 1, kernel_size=(1, 1), padding=(0, 0), stride=(1, 1))

    def forward(self, x, idx):
        adj = self.graph_constructor(idx)
        x = self.mixprop(x, adj)
        x = self.dilated_inception(x)
        x = self.output_layer(x)
        return x

# Load the METR-LA dataset
loader = METRLADatasetLoader()
dataset = loader.get_dataset()
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

# Initialize model, optimizer, and loss function
num_nodes = dataset[0].x.shape[0]
input_dim = dataset[0].x.shape[2]
output_dim = 32
gdep = 2
dropout = 0.3
alpha = 0.05
dilation_factor = 2

model = CustomMTGNN(num_nodes=num_nodes, input_dim=input_dim, output_dim=output_dim, gdep=gdep, dropout=dropout,
                    alpha=alpha, dilation_factor=dilation_factor).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
num_epochs = 10  # Set to a smaller number for testing
# Training loop with input reshaping
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for snapshot in train_dataset:
        optimizer.zero_grad()

        # Reshape input x to match model expectations
        x = snapshot.x.permute(2, 0, 1).unsqueeze(0).to(device)  # Shape: [1, features, nodes, time_steps]
        idx = torch.arange(num_nodes).to(device)
        y = snapshot.y.unsqueeze(0).to(device)  # Add batch dimension

        # Forward pass
        output = model(x, idx)

        # Compute loss
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch {epoch + 1}, Train Loss: {total_loss / len(train_dataset):.4f}')
